In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,VotingClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, train_test_split,GridSearchCV,StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

In [2]:
train_data=pd.read_csv('train.csv',low_memory=False)

In [3]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
#train_data.head(60)
display_all(train_data.head(15))

,surveyid,village,survey_date,femaleres,age,married,children,hhsize,edu,hh_children,hh_totalmembers,cons_nondurable,asset_livestock,asset_durable,asset_phone,asset_savings,asset_land_owned_total,asset_niceroof,cons_allfood,cons_ownfood,cons_alcohol,cons_tobacco,cons_med_total,cons_med_children,cons_ed,cons_social,cons_other,ent_wagelabor,ent_ownfarm,ent_business,ent_nonagbusiness,ent_employees,ent_nonag_revenue,ent_nonag_flowcost,ent_farmrevenue,ent_farmexpenses,ent_animalstockrev,ent_total_cost,fs_adskipm_often,fs_adwholed_often,fs_chskipm_often,fs_chwholed_often,fs_meat,fs_enoughtom,fs_sleephun,med_expenses_hh_ep,med_expenses_sp_ep,med_expenses_child_ep,med_portion_sickinjured,med_port_sick_child,med_afford_port,med_sickdays_hhave,med_healthconsult,med_vacc_newborns,med_child_check,med_u5_deaths,ed_expenses,ed_expenses_perkid,ed_schoolattend,ed_sch_missedpc,ed_work_act_pc,labor_primary,wage_expenditures,durable_investment,nondurable_investment,given_mpesa,amount_given_mpesa,received_mpesa,amount_received_mpesa,net_mpesa,saved_mpesa,amount_saved_mpesa,early_survey,depressed,day_of_week
0,926,91,23-Nov-61,1,28.0,1,4,6,10,0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0.000000,0.000000,0.0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.00000,0.000000,0,0.0,0,0.000000,0.000000,1,0.000000,0,0,5
1,747,57,24-Oct-61,1,23.0,1,3,5,8,0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0.000000,0.000000,0.0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.00000,0.000000,0,0.0,1,4.804611,4.804611,0,0.000000,0,1,3
2,1190,115,05-Oct-61,1,22.0,1,3,5,9,0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0.000000,0.000000,0.0,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.00000,0.000000,0,0.0,0,8.007685,8.007685,1,0.000000,0,0,5
3,1065,97,23-Sep-61,1,27.0,1,2,4,10,2,4.0,378.832920,526.671080,196.989040,24.023054,49.647648,0.0,0,289.173520,36.434967,0.0,0.000000,4.163996,0.960922,1.681614,39.771500,44.042267,0,1,0,1,0,32.030739,0.000000,22.288055,1.875133,16.282293,17.890503,0.0,0.0,0.0,0.0,3.0,1.0,1.0,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,NaN,1,1,NaN,20.179367,10.089684,0.500000,1.500000,0.000000,0,0,778.11230,69.219765,0,0.0,0,0.000000,0.000000,1,1.249199,0,0,0
4,806,42,12-Sep-61,0,59.0,0,4,6,10,4,6.0,258.671880,8.260629,173.526540,19.218445,0.000000,0.5,0,173.010990,0.870168,0.0,0.000000,0.800768,0.800768,2.268844,8.087762,74.503502,1,0,0,0,0,0.000000,0.000000,0.533846,2.073101,0.000000,2.073101,0.0,0.0,0.0,0.0,2.0,0.0,0.0,NaN,NaN,1.233183,0.333333,0.250000,0.500000,1.166667,1.0,1,1,NaN,27.226130,6.806532,0.750000,1.750000,1.250000,1,0,201.00562,4.341945,0,0.0,0,0.000000,0.000000,0,0.000000,0,0,3
5,483,25,08-Sep-61,1,35.0,1,6,8,10,6,8.0,63.790752,359.374660,73.670700,0.000000,0.000000,1.7,0,40.938339,2.268844,NaN,5.582501,0.000000,0.000000,2.669228,3.069613,11.531066,0,1,0,1,0,4.804611,4.804611,2.268844,0.189070,0.000000,4.993681,7.5,3.0,7.5,0.0,1.0,1.0,1.0,1.601537,NaN,NaN,0.125000,0.000000,1.000000,0.875000,0.0,1,1,NaN,32.030739,6.406148,0.800000,0.000000,1.500000,0,0,444.25610,7.662910,0,0.0,0,0.000000,0.000000,0,0.000000,0,0,6
6,849,130,31-Oct-61,0,34.0,0,1,3,9,1,3.0,60.614365,4.130314,219.250410,0.000000,0.000000,1.0,0,48.963181,0.266923,0.0,0.000000,0.160154,0.160154,0.533846,0.066731,10.890451,0,0,0,0,0,0.000000,0.000000,0.266923,0.022244,0.000000,0.022244,20.0,3.0,20.0,0.0,1.0,0.0,1.0,0.053385,NaN,0.160154,1.000000,1.000000,0.333333,0.000000,0.0,1,1,NaN,6.406148,6.406148,1.000000,0.000000,1.000000,0,0,225.62288,0.556089,0,0.0,0,0.000000,0.

In [4]:
train_data.fillna(0,inplace=True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 75 columns):
surveyid                   1143 non-null int64
village                    1143 non-null int64
survey_date                1143 non-null object
femaleres                  1143 non-null int64
age                        1143 non-null float64
married                    1143 non-null int64
children                   1143 non-null int64
hhsize                     1143 non-null int64
edu                        1143 non-null int64
hh_children                1143 non-null int64
hh_totalmembers            1143 non-null float64
cons_nondurable            1143 non-null float64
asset_livestock            1143 non-null float64
asset_durable              1143 non-null float64
asset_phone                1143 non-null float64
asset_savings              1143 non-null float64
asset_land_owned_total     1143 non-null float64
asset_niceroof             1143 non-null int64
cons_allfood               114

In [5]:
train_data['survey_date'] = pd.to_datetime(train_data['survey_date'],infer_datetime_format=True)
train_data['month'] = train_data['survey_date'].dt.month
train_data['year'] = train_data['survey_date'].dt.year
train_data.drop(['survey_date'],axis=1,inplace=True)

In [6]:
x= train_data.drop(['surveyid','depressed'],axis=1)
scaler = MinMaxScaler()
x_scale = scaler.fit_transform(x)
y= train_data.depressed
display_all(pd.DataFrame(x_scale).head(15))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73
0,0.309278,1.0,0.148649,1.0,0.4,0.454545,0.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.312500,1.0,0.000000,0.0,0.833333,0.875,1.0
1,0.192440,1.0,0.081081,1.0,0.3,0.363636,0.388889,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,0.013636,0.321875,0.0,0.000000,0.0,0.500000,0.750,1.0
2,0.391753,1.0,0.067568,1.0,0.3,0.363636,0.444444,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.022727,0.328125,1.0,0.000000,0.0,0.833333,0.750,1.0
3,0.329897,1.0,0.135135,1.0,0.2,0.272727,0.500000,0.2,0.333333,0.264620,0.191202,0.052949,0.125000,0.022143,0.000000,0.0,0.208494,0.200662,0.0,0.000000,0.020155,0.004651,0.0126,0.283810,0.152346,0.0,1.0,0.0,1.0,0.0,0.004167,0.000000,0.138131,0.049269,0.092424,0.008653,0.000,0.000,0.000,0.0,0.136364,1.0,1.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0126,0.054000,0.333333,0.050000,0.000000,0.0,0.0,0.205723,0.030420,0.0,0.0,0.0,0.000000,0.312500,1.0,0.002557,0.0,0.000000,0.625,1.0
4,0.140893,0.0,0.567568,0.0,0.4,0.454545,0.500000,0.4,0.500000,0.180686,0.002999,0.046642,0.100000,0.000000,0.018519,0.0,0.124741,0.004792,0.0,0.000000,0.003876,0.003876,0.0170,0.057714,0.257714,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.003309,0.054471,0.000000,0.001003,0.000,0.000,0.000,0.0,0.090909,0.0,0.0,0.000000,0.000000,0.00385,0.166667,0.250000,0.500000,0.037634,1.0,0.0,0.0,0.0,0.0170,0.036429,0.500000,0.058333,0.250000,1.0,0.0,0.053143,0.001908,0.0,0.0,0.0,0.000000,0.312500,0.0,0.000000,0.0,0.500000,0.625,1.0
5,0.082474,1.0,0.243243,1.0,0.6,0.636364,0.500000,0.6,0.666667,0.044559,0.130467,0.019802,0.000000,0.000000,0.062963,0.0,0.029516,0.012495,0.0,0.133333,0.000000,0.000000,0.0200,0.021905,0.039887,0.0,1.0,0.0,1.0,0.0,0.000625,0.002324,0.014061,0.004968,0.000000,0.002415,0.375,0.150,0.375,0.0,0.045455,1.0,1.0,0.012898,0.000000,0.00000,0.062500,0.000000,1.000000,0.028226,0.0,0.0,0.0,0.0,0.0200,0.034286,0.533333,0.000000,0.300000,0.0,0.0,0.117456,0.003368,0.0,0.0,0.0,0.000000,0.312500,0.0,0.000000,0.0,1.000000,0.625,1.0
6,0.443299,0.0,0.229730,0.0,0.1,0.181818,0.444444,0.1,0.250000,0.042340,0.001499,0.058932,0.000000,0.000000,0.037037,0.0,0.035302,0.001470,0.0,0.000000,0.000775,0.000775,0.0040,0.000476,0.037671,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.001654,0.000584,0.000000,0.000011,1.000,0.150,1.000,0.0,0.045455,0.0,1.0,0.000430,0.000000,0.00050,0.500000,1.000000,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0040,0.034286,0.666667,0.000000,0.200000,0.0,0.0,0.059652,0.000244,0.0,0.0,0.0,0.000000,0.312500,0.0,0.000000,0.0,0.500000,0.750,1.0
7,0.243986,1.0,0.054054,1.0,0.2,0.272727,0.500000,0.2,0.333333,0.082474,0.043614,0.054628,0.350000,0.021429,0.000000,0.0,0.078277,0.033591,0

In [7]:
loss = ['deviance', 'exponential']

# Number of trees used in the boosting process
n_estimators = [500, 900]

# Maximum depth of each tree
max_depth = [10, 15]

# Minimum number of samples per leaf
min_samples_leaf = [14]

# Minimum number of samples to split a node
min_samples_split = [10]

# Maximum number of features to consider for making splits
max_features = ['auto', 'sqrt', 'log2', None]

# Define the grid of hyperparameters to search
# 'loss': loss,
hyperparameter_grid = {
                       'n_estimators': n_estimators,
                       'max_depth': max_depth,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}
param_dict={'max_depth': [2, 3, 5, 10, 15],
                             'subsample': [0.25,0.5,0.6,0.75],
                             'colsample_bytree': [0.25,0.5,0.6,0.75]
        }

In [8]:
#g_model = GradientBoostingClassifier(random_state=42, criterion="mae")
r_model =  RandomForestClassifier(n_estimators=500,random_state=42)
#gr_model = GridSearchCV(estimator=r_model,
#                              param_grid=hyperparameter_grid,
#                              cv=4, 
#                            n_jobs = -1, verbose = 1)
ra_model = RandomizedSearchCV(estimator=r_model,
                               param_distributions=hyperparameter_grid,
                               cv=4, n_iter=10, 
                               scoring = 'neg_mean_absolute_error',
                               n_jobs = -1, verbose = 1, 
                               random_state=42)
x_model =  xgb.XGBClassifier(random_state=42,learning_rate=0.01)
model   =   VotingClassifier(estimators=[('rf', ra_model), ('xg', x_model)],voting='hard')


In [9]:
train_x,test_x,train_y,test_y=train_test_split(x_scale,y,random_state=0)


In [10]:
model.fit(train_x,train_y)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.0min finished


VotingClassifier(estimators=[('rf', RandomizedSearchCV(cv=4, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
     ...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [11]:
pred_train_set = model.predict(test_x)
print(mean_absolute_error(pred_train_set,test_y))

0.15034965035


In [12]:
test_data=pd.read_csv('test.csv',low_memory=False)

In [13]:
test_data.fillna(0 ,inplace=True)
#test_data.replace({'.d':0})

In [14]:
test_data['survey_date'] = pd.to_datetime(test_data['survey_date'],infer_datetime_format=True)
test_data['month'] = test_data['survey_date'].dt.month
test_data['year'] = test_data['survey_date'].dt.year
test_data.drop(['survey_date'],axis=1,inplace=True)

In [15]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 76 columns):
surveyid                   286 non-null int64
village                    286 non-null int64
femaleres                  286 non-null int64
age                        286 non-null float64
married                    286 non-null int64
children                   286 non-null int64
hhsize                     286 non-null int64
edu                        286 non-null int64
hh_children                286 non-null int64
hh_totalmembers            286 non-null float64
cons_nondurable            286 non-null float64
asset_livestock            286 non-null float64
asset_durable              286 non-null float64
asset_phone                286 non-null float64
asset_savings              286 non-null float64
asset_land_owned_total     286 non-null float64
asset_niceroof             286 non-null int64
cons_allfood               286 non-null float64
cons_ownfood               286 non-null float64
c

In [16]:
x_test=test_data.drop(['surveyid','depressed'],axis=1)
x_test_scale = scaler.transform(x_test)
depression = model.predict(x_test_scale)

In [18]:
d={'surveyid' :test_data['surveyid'], 'depressed':depression}
prediction =pd.DataFrame(d)
prediction.to_csv('prediction.csv' ,index=False)

In [ ]:
#pd.DataFrame(pred_train_set).to_csv('traa.csv',index=False)